# Bring in Data

In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
import types
import pandas as pd
df = pd.read_csv("Event.csv")
df.head()

,Opportunity,Time,Stage,Action
0,OP001,43456.60550,1,Emailed Client
1,OP001,43466.19231,1,Emailed Client
2,OP001,43471.51592,2,Meeting with Client
3,OP001,43479.39509,3,Presented to Client
4,OP001,43483.32009,3,Presented to Client


In [3]:
df['Event Key'] = df['Stage'].astype(str) +'-'+ df['Action']

In [4]:
events = list(df['Event Key'].unique())
ev = events.pop(8)
events.append(ev)
events = ['0-Dummy'] + events

In [5]:
events

['0-Dummy',
 '1-Emailed Client',
 '2-Meeting with Client',
 '3-Presented to Client',
 '4-Prepared Custom Demonstration',
 '5-Prepared POC',
 '6-Provided Pre-Sales Support',
 '8-Prepared Commercials',
 '9-Negotiation Meeting',
 '7-Prepared RFP Responses',
 '10-Commercials Signed',
 '11-Post Sales Support']

In [6]:
df.head()

,Opportunity,Time,Stage,Action,Event Key
0,OP001,43456.60550,1,Emailed Client,1-Emailed Client
1,OP001,43466.19231,1,Emailed Client,1-Emailed Client
2,OP001,43471.51592,2,Meeting with Client,2-Meeting with Client
3,OP001,43479.39509,3,Presented to Client,3-Presented to Client
4,OP001,43483.32009,3,Presented to Client,3-Presented to Client


# Prepare Data

In [7]:
series_length = 5
categories = len(events)
time_steps = series_length -1 

In [8]:
df.set_index('Opportunity', inplace=True)

In [9]:
history = []
for op in df.index:
    for x in range(len(df['Stage'].loc[op].values)-series_length):
        history.append(df['Stage'].loc[op].values[x:x+series_length])

In [10]:
history[:5]

[array([1, 1, 2, 3, 3]),
 array([1, 2, 3, 3, 3]),
 array([2, 3, 3, 3, 4]),
 array([3, 3, 3, 4, 5]),
 array([3, 3, 4, 5, 5])]

In [11]:
from tensorflow.keras.utils import to_categorical

In [12]:
X= np.array([row[:time_steps] for row in history])

In [13]:
X = to_categorical(X)

In [14]:
y=np.array([row[time_steps] for row in history])

In [15]:
y = to_categorical(y).reshape(-1,categories)

In [16]:
X.shape

(16176, 4, 12)

In [17]:
X

array([[[0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]]], dtype=float32)

In [18]:
y.shape

(16176, 12)

# Create Train and Test Set

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3)

In [21]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(11323, 4, 12) (4853, 4, 12) (11323, 12) (4853, 12)


# Modelling

In [22]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [23]:
model = Sequential()

In [24]:
model.add(LSTM(10, activation='relu', return_sequences=True, input_shape=(time_steps,categories)))
model.add(LSTM(10, activation='relu'))
model.add(Dense(categories, activation='softmax'))
model.compile('adam', loss='categorical_crossentropy')

In [25]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 4, 10)             920       
                                                                 
 lstm_1 (LSTM)               (None, 10)                840       
                                                                 
 dense (Dense)               (None, 12)                132       
                                                                 
Total params: 1,892
Trainable params: 1,892
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit(X_train,y_train, epochs=10)

Epoch 1/10
354/354 [==============================] - 5s 5ms/step - loss: 1.8159
Epoch 2/10
354/354 [==============================] - 2s 5ms/step - loss: 1.1533
Epoch 3/10
354/354 [==============================] - 2s 5ms/step - loss: 1.0625
Epoch 4/10
354/354 [==============================] - 3s 8ms/step - loss: 1.0160
Epoch 5/10
354/354 [==============================] - 2s 6ms/step - loss: 0.9926
Epoch 6/10
354/354 [==============================] - 2s 5ms/step - loss: 0.9743
Epoch 7/10
354/354 [==============================] - 2s 7ms/step - loss: 0.9586
Epoch 8/10
354/354 [==============================] - 2s 6ms/step - loss: 0.9512
Epoch 9/10
354/354 [==============================] - 2s 6ms/step - loss: 0.9470
Epoch 10/10
354/354 [==============================] - 3s 8ms/step - loss: 0.9407


# Generate Predictions

In [27]:
prediction = model.predict(X_test)

152/152 [==============================] - 1s 2ms/step


In [28]:
# Next event is
events[np.argmax(prediction[1])]

'5-Prepared POC'

In [29]:
np.argmax(y_test[1])

6

In [30]:
np.argmax(prediction[1])

5

# Evaluate Accuracy

In [31]:
import tensorflow as tf
from tensorflow.keras.metrics import top_k_categorical_accuracy

In [32]:
print('Correct event was in top prediction:', top_k_categorical_accuracy(y_test,prediction, k=1).numpy().sum()/top_k_categorical_accuracy(y_test,prediction, k=2).numpy().size)
print('Correct event was in top two prediction:', top_k_categorical_accuracy(y_test,prediction, k=2).numpy().sum()/top_k_categorical_accuracy(y_test,prediction, k=3).numpy().size)

Correct event was in top prediction: 0.5751081805069029
Correct event was in top two prediction: 0.8932618998557593


In [33]:
from ipywidgets import widgets
lbl_act_1 = widgets.Label('select first activity in sequence from the dropdown')
display(lbl_act_1)
w_act_1 = widgets.Dropdown(
    options=['0-Dummy',
 '1-Emailed Client',
 '2-Meeting with Client',
 '3-Presented to Client',
 '4-Prepared Custom Demonstration',
 '5-Prepared POC',
 '6-Provided Pre-Sales Support',
 '8-Prepared Commercials',
 '9-Negotiation Meeting',
 '7-Prepared RFP Responses',
 '10-Commercials Signed',
 '11-Post Sales Support'],
    value='0-Dummy',
    description='Activity_1:',
)

def on_change_act_1(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global act_1_val
        act_1_val = change['new']

w_act_1.observe(on_change_act_1)

display(w_act_1)

Label(value='select first activity in sequence from the dropdown')

Dropdown(description='Activity_1:', options=('0-Dummy', '1-Emailed Client', '2-Meeting with Client', '3-Presen…

In [34]:
lbl_act_2 = widgets.Label('select second activity in sequence from the dropdown')
display(lbl_act_2)
w_act_2 = widgets.Dropdown(
    options=['0-Dummy',
 '1-Emailed Client',
 '2-Meeting with Client',
 '3-Presented to Client',
 '4-Prepared Custom Demonstration',
 '5-Prepared POC',
 '6-Provided Pre-Sales Support',
 '8-Prepared Commercials',
 '9-Negotiation Meeting',
 '7-Prepared RFP Responses',
 '10-Commercials Signed',
 '11-Post Sales Support'],
    value='0-Dummy',
    description='Activity_2:',
)

def on_change_act_2(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global act_2_val
        act_2_val = change['new']

w_act_2.observe(on_change_act_2)

display(w_act_2)

Label(value='select second activity in sequence from the dropdown')

Dropdown(description='Activity_2:', options=('0-Dummy', '1-Emailed Client', '2-Meeting with Client', '3-Presen…

In [35]:
lbl_act_3 = widgets.Label('select third activity in sequence from the dropdown')
display(lbl_act_3)
w_act_3 = widgets.Dropdown(
    options=['0-Dummy',
 '1-Emailed Client',
 '2-Meeting with Client',
 '3-Presented to Client',
 '4-Prepared Custom Demonstration',
 '5-Prepared POC',
 '6-Provided Pre-Sales Support',
 '8-Prepared Commercials',
 '9-Negotiation Meeting',
 '7-Prepared RFP Responses',
 '10-Commercials Signed',
 '11-Post Sales Support'],
    value='0-Dummy',
    description='Activity_3:',
)

def on_change_act_3(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global act_3_val
        act_3_val = change['new']

w_act_3.observe(on_change_act_3)

display(w_act_3)

Label(value='select third activity in sequence from the dropdown')

Dropdown(description='Activity_3:', options=('0-Dummy', '1-Emailed Client', '2-Meeting with Client', '3-Presen…

In [36]:
lbl_act_4 = widgets.Label('select fourth activity in sequence from the dropdown')
display(lbl_act_4)
w_act_4 = widgets.Dropdown(
    options=['0-Dummy',
 '1-Emailed Client',
 '2-Meeting with Client',
 '3-Presented to Client',
 '4-Prepared Custom Demonstration',
 '5-Prepared POC',
 '6-Provided Pre-Sales Support',
 '8-Prepared Commercials',
 '9-Negotiation Meeting',
 '7-Prepared RFP Responses',
 '10-Commercials Signed',
 '11-Post Sales Support'],
    value='0-Dummy',
    description='Activity_4:',
)

def on_change_act_4(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global act_4_val
        act_4_val = change['new']

w_act_4.observe(on_change_act_4)

display(w_act_4)

Label(value='select fourth activity in sequence from the dropdown')

Dropdown(description='Activity_4:', options=('0-Dummy', '1-Emailed Client', '2-Meeting with Client', '3-Presen…

In [37]:
act_lst = [int(act_1_val[0]), int(act_2_val[0]), int(act_3_val[0]), int(act_4_val[0])]

In [38]:
act_lst

[1, 2, 1, 3]

In [39]:
act_arr = np.array([act_lst])

In [40]:
act_arr.shape

(1, 4)

In [41]:
to_categorical(act_arr, 12)

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [42]:
to_categorical(act_arr, 12).shape

(1, 4, 12)

In [43]:
model.predict(to_categorical(act_arr, 12))

1/1 [==============================] - 0s 29ms/step


array([[1.4173084e-10, 1.3492786e-08, 2.3497188e-02, 6.9493222e-01,
        2.4575253e-01, 2.7935946e-02, 5.8520664e-03, 1.5678358e-03,
        1.7450933e-05, 2.2094324e-05, 3.3178661e-04, 9.0820969e-05]],
      dtype=float32)

In [52]:
lst_val = model.predict(to_categorical(act_arr, 12)).tolist()

1/1 [==============================] - 0s 23ms/step


In [58]:
for i  in range(0,12):
  print( str(i)+" action probability "+str(lst_val[0][i]*100))

0 action probability 1.4173083540125475e-08
1 action probability 1.3492786443691784e-06
2 action probability 2.3497188463807106
3 action probability 69.4932222366333
4 action probability 24.57525283098221
5 action probability 2.7935946360230446
6 action probability 0.5852066446095705
7 action probability 0.156783580314368
8 action probability 0.0017450933228246868
9 action probability 0.0022094323867349885
10 action probability 0.033178660669364035
11 action probability 0.009082096948986873


In [59]:
events[np.argmax(model.predict(to_categorical(act_arr, 12)))]

1/1 [==============================] - 0s 36ms/step


'3-Presented to Client'